## 引子：
最近在用python脚本整理IP库时遇到了一个很奇怪的问题，详情描述如下：

现有文本内容格式为：
<pre>
apnic|JP|ipv6|2001:200:2000::|35|20030423|allocated
apnic|JP|ipv6|2001:200:4000::|34|20030423|allocated
apnic|JP|ipv6|2001:200:8000::|33|20030423|allocated
</pre>

欲将其格式转换成IPV6数据库的既有格式：
<pre>begin_ip_num,end_ip_num,cidr,country,update_time</pre>
故有python脚本：

In [94]:
# -*- coding: utf-8 -*-

import fileinput
import re
from iptools import ipv6

IPV6_RECORD = re.compile(r"\|[A-Z][A-Z]\|ipv6\|")

COUNTRY = {
    "JP": u"日本",
    "CN": u"中国",
    #...
}

try:
    for line in fileinput.input("delegate"):
        if IPV6_RECORD.search(line):
            fields = line.split("|")
            cidr = "{0[3]}/{0[4]}".format(fields)
            block = ipv6.cidr2block(cidr)

            print "{},{},{},{},{}".format(
            #print u"{},{},{},{},{}".format(
                ipv6.ip2long(block[0]),
                ipv6.ip2long(block[1]),
                cidr,
                COUNTRY[fields[1]],
                fields[5]
            )
finally:
    fileinput.close()

UnicodeEncodeError: 'ascii' codec can't encode characters in position 0-1: ordinal not in range(128)

以上错误出现的原因是什么呢？且听我一一道来。

----

## str 与 unicode
稍熟悉python的人都知道python中有两种字符串类型：str和unicode

In [96]:
# -*- coding: utf-8 -*-

s = "昔我往矣，杨柳依依。今我来思，雨雪霏霏。"
u = u"昔我往矣，杨柳依依。今我来思，雨雪霏霏。"

# print type of s and u
print type(s)
print type(u)

<type 'str'>
<type 'unicode'>


两个python字符串类型间可以用 encode / decode 方法转换：

In [97]:
# from str to unicode
print s.decode(encoding="utf-8")
               
#print s.decode(encoding="gbk")

# from unicode to str
print u.encode(encoding="utf-8")

昔我往矣，杨柳依依。今我来思，雨雪霏霏。
昔我往矣，杨柳依依。今我来思，雨雪霏霏。


在上面的代码中，有两个问题必须要说明一下的：
1. 在从str转unicode的函数中encoding为什么是utf-8，其他编码可不可以呢？
2. 按照常理，从str转unicode应该是编码(encode)，从unicode转str应该是解码(decode)，而在python中却是反过来的呢？

答案：

1. 指定其他编码是不可以，因为str的编码格式已经由代码首行的
```python
# -*- coding: utf-8 -*-
```
指定为utf-8，使用其他的编码格式都会出现错误。读者不妨将上面代码块中
```python
#print s.decode(encoding="gbk)
```
的注释取消，重新执行一下试试。
2. python认为16位的unicode才是字符的唯一内码，而大家常用的字符集如gb2312，gbk，utf-8，以及ascii都是字符的二进制编码形式。把字符从 unicode 转换成二进制编码，当然是要 encode。反过来，在python中出现的str都是用开发者指定的，python并不知道str的编码，所以需要由开发者指定正确的字符集来decode。

综上所述：**python将unicode作为字符的唯一内码，所以在进行同时包含str与unicode的运算时，python总是把str转换成unicode再运算，当然，运算结果也都是 unicode**。请看下面示例：

In [98]:
print s + u

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe6 in position 0: ordinal not in range(128)

在上面的运算中，包含了str和unicode两种不同的字符串类型，所以需要对s进行decode转换为unicode再进行连接运算。从运行结果中的UnicodeDecodeError信息中，我们可以看出这样转换过程的确进行了，错误的原因也是很容易推断的 -- 在对s解码是使用了错误的ascii格式。那么，为什么系统会采用ascii编码而不是其他的编码格式呢？
由于python不知道str的编码格式，所以python只能采用系统默认的编码格式。

In [99]:
import sys

coding = sys.getdefaultencoding()
print "the system 's default encoding is {}".format(coding)

# demonstrate the UnicodeDecodeError above
s.decode(encoding=coding)

the system 's default encoding is ascii


UnicodeDecodeError: 'ascii' codec can't decode byte 0xe6 in position 0: ordinal not in range(128)

In [ ]:
import sys

reload(sys)
sys.setdefaultencoding("utf-8")

print s + u

**注意：虽然通过上面的方式可以解决问题，但是这种方式却是不推荐的，因为sys.setdefaultencoding的效果是全局性的，一个大的项目可能由多个文件组成，用这种方式只能是一波刚平，一波又起，使问题更加复杂。**

----

## python encoding的常见陷阱
掌握了上面的知识后，我们来谈谈使用python过程中关于encoding的常见陷阱。

**陷阱1：混合运算**

所谓混合运算就是指上面所述的同时包含str和unicode两种字符串类型时的运算，陷阱产生在str到unicode的隐式转换过程中。也许你要说向s+u这样运算我们并不常做，其实它还有两种更常见的形式：% 和 format进行的字符串格式化。请看下面示例：

In [ ]:
# trap demo
print "诗经：%s" % u
print u"诗经：%s" % s
print "诗经：{}".format(u)
print u"诗经：{}".format(s)

# correct demo
# print "诗经：%s" % s
# print u"诗经：%s" % u
# print  "诗经：{}".format(s)
# print u"诗经：{}".format(u)

上面所示的每一代码都会产生又编码问题导致的错误，正确使用使用方式如注释部分所示。

```python
print "诗经：%s" % u
```
错误产生于```"诗经：%s".decode(encoding="ascii")```的转换过程；
```python
print u"诗经：%s" % s
```
错误产生于```s.decode(encoding="ascii")```的转换过程。使用format函数错误产生原因可类推，不再赘述。

此时此刻，终于到了该解决我们引子中遇到问题的时候了。显然在```COUNTRY[fields[1]]```的结果是unicode字符串```u"日本"```，而格式化字符串```"{},{},{},{},{}"```是str类型，因此我们落入了混合运算的陷阱。解决方式很简单，只需要将格式化字符串改写为```u"{},{},{},{},{}"```即可，读者可在代码块中更改后，```ctrl+enter```重新执行。

**陷阱二：管道、子进程、重定向**

在介绍这个陷阱前，首先我们需要了解print函数，与所有语言一样，这个python命令实际上是把字符打印到标准输出流--sys.stdout。而不同的是python在这里变了个魔术，它会按照sys.stdout.encoding来给unicode编码，而把str直接输出，扔给操作系统去解决。通常情况，sys.stdout.encoding的值与linux的$LANG环境变量保持一致。

接下来看一个python程序encoding_test.py，程序代码如下所示：

```python
# -*- coding: utf-8 -*-

if __name__ == "__main__":
    import sys
    print "The stdout's encoding is {}.".format(sys.stdout.encoding)
    
    print "昔我往矣，杨柳依依。今我来思，雨雪霏霏。"
    print u"昔我往矣，杨柳依依。今我来思，雨雪霏霏。"
    
```

In [1]:
!python encoding_test.py

python: can't open file 'encoding_test.py': [Errno 2] No such file or directory


In [ ]:
!python encoding_test.py | more

针对上面的运行结果，有两个问题：
1. 为什么程序在通过管道后，发生了UnicodeEncodeError错误？
2. 为什么正确显示的是str字符而不是unicode字符？

关于问题1：也就是我们要描述的陷阱2的内容，**在python程序通过管道、子进程、重定向方式运行时，sys.stdout.encoding就会失效。**这一点在程序两次的执行的输出结果```The stdout's encoding is UTF-8.```和```The stdout's encoding is None.```可以得到证明。

关于问题2：了解到陷阱2的内容后，问题2的内容也就自然清楚了。在程序通过管道执行后，sys.stdout.encoding失效，print函数只能通过系统默认的ascii来对unicode进行编码。因为ascii不能表示中文字符，必然会发生以上错误。

----

##指导原则：
- 在代码中通过```# -*- coding: utf-8 -*-```指定文件中str的编码；
- 在包含中文时，使用unicode字符串；
- 使用python管道、子进程、重定向时，注意编码问题。